In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd       # importing the libraries we will need
import numpy as np

##### In this cell, we create the list of URLs. Some of the next cells have the # before each line because there is no need to execute that cell more than once  #####

In [2]:
#number_of_pages=300
#for page in range(1,number_of_pages+1):
    #url="https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(page)
    #cnt = requests.get(url)
    #soup = BeautifulSoup(cnt.content, features="lxml")
    #links = soup.find_all('a',href=True)
    #f = open("book_list.txt",'a')
    #n=0
    #for link in links:
        #fullLink = link.get('href')
        #if fullLink[0:11]=="/book/show/":
            #n=n+1
            #if n%2==0:
                #fullLink="https://www.goodreads.com/"+fullLink+"\n"
                #f.write(fullLink)
    #f.close()

        

In [3]:
#!pip3 install selenium
#!pip3 install webdriver_manager
#from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
#from webdriver_manager.firefox import GeckoDriverManager
#driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

##### In this cell, we get the html file relative to each URL.  #####

In [4]:
#f=open("book_list.txt","r")
#n=0 
#done=29648
#for url in f:
    #n=n+1
    #if n>done:
        #driver.get(url)
        #x=driver.page_source
        #f = open("book"+str(n)+".html", "w",encoding="utf-8")
        #f.write(x)
        #f.close()
#f.close()

##### In this cell, we create a function which analyzes every html file and returns the features we are looking for. For some books, some features may be missing, so we used the try-except statement in order to avoid errors #####

In [5]:
def scrap_book(html):
    page_soup =BeautifulSoup(html,"html.parser") 
    try:
        bookTitle = page_soup.find_all('h1')[0].contents[0].replace('\n', '').strip()
    except:
        bookTitle=""
    try:
        bookSeries = page_soup.find("h2", id="bookSeries")
        bookSeries=str(bookSeries).split("\n")[2].strip()
    except:
        bookSeries="" 
    try:
        bookAuthors = page_soup.find_all('span', itemprop='name')[0].contents[0]
    except: 
        bookAuthors=""
    try:
        ratingValue = page_soup.find_all('span', itemprop='ratingValue')[0].contents[0].strip()
    except:
        ratingValue=""
    try:
        NumberofPages = page_soup.find_all('span', itemprop='numberOfPages')[0].contents[0].split(" ")[0]
    except:
        NumberofPages=""
    try:
        Plot = ' '.join([c for c in page_soup.find_all('div', id="description")[0].contents[1].contents if isinstance(c, str)])
    except:
        Plot=""
    try:
        Characters = []
        for a in page_soup.find_all('a', href=True): 
            if a is not None:
                b=str(a)
                if "/characters/" in b:
                    Ch_name=str()
                    c=a['href']
                    d=c.split("-")
                    d=d[1:]
                    for i in d:
                        Ch_name=Ch_name+i+" "
                    Characters.append(Ch_name)
    except:
        Characters=""
    try:
        Setting = []
        for a in page_soup.find_all('a', href=True): 
            if a is not None:
                b=str(a)
                if "/places/" in b:
                    Setting_name=str()
                    c=a['href']
                    d=c.split("-")
                    d=d[1:]
                    for i in d:
                        Setting_name=Setting_name+i+" "
                    Setting.append(Setting_name)
    except:
        Setting=""
    try:
        Url=page_soup.find('link', href=True,rel="canonical")["href"]
    except:
        Url=""
    try:
        ratings = page_soup.find_all('a', href="#other_reviews")
        ratingCount = -1
        reviewCount = -1
        for raiting in ratings:
            if raiting.find_all('meta', itemprop="ratingCount"):
                ratingCount = raiting.text.replace('\n', '').strip().split(' ')[0]
            elif raiting.find_all('meta', itemprop="reviewCount"):
                reviewCount = raiting.text.replace('\n', '').strip().split(' ')[0]
    except:
        ratingCount=""
        reviewCount=""
    return (bookTitle, bookSeries, bookAuthors,ratingValue, ratingCount, reviewCount,Plot,NumberofPages,Characters,Setting,Url)

##### In this cell, we apply the scrap_book function to every book. The results are saved in a .tsv file. In this cell, we forgot to write the line relative to the indexes. We will solve this problem later #####

In [6]:
#for i in range(1,30001):
    #try:
        #s="html/book"+str(i)+".html"
        #book=open(s,"r",encoding="utf-8")
        #html=book.read()
        #scrap_book(html)
        #results=scrap_book(html)
        #toWrite=""
        #for elem in results:
            #toWrite=toWrite+str(elem)+" \t "
        #name="article"+str(i)+".tsv"
        #f =open(name,'w',encoding="utf-8")
        #f.write(toWrite)
        #f.close()
        #book.close()
    #except:
        #pass


In [7]:
#!pip3 install langdetect

In [2]:
from langdetect import detect
import os

##### In this cell, we remove the books which have the plot written in a language different than English. We also realize that we forgot to write the indexes in the tsv files, so the data became the indexes. We are still able to use the data correctly, but we want to add the indexes too. So we fix this problem in the next cell. #####

In [9]:
#for i in range(1,30001):
    #try:
        #s="tsv/article"+str(i)+".tsv"
        #book=pd.read_csv(s,sep="\t")
        #if detect(book.columns[6]) != "en":
            #os.remove(s)
    #except:
        #pass

##### In this cell, we write new .tsv files where the indexes are included. We also delete the old .tsv files #####

In [10]:
#for i in range(1,30001):
    #try:
        #s="tsv/article"+str(i)+".tsv"
        #book=open(s,"r",encoding="utf-8")
        #data=book.read()
        #book.close()
        #s2="tsv/article_"+str(i)+".tsv"
        #f=open(s2,'w',encoding="utf-8")
        #f.write("bookTitle\tbookSeries\tbookAuthors\tratingValue\tratingCount\treviewCount\tPlot\tNumberofPages\tCharacters\tSetting\tUrl\t\n")
        #f.write(data)
        #f.close()
        #os.remove(s)
    #except:
        #pass

In [3]:
#!pip3 install nltk
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aless_oh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aless_oh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aless_oh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aless_oh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aless_oh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

True

##### In this cell, I create the vocabulary.csv file, in which every word will be mapped to an integer "term_id" #####

In [12]:
#f = open("vocabulary.csv",'w',encoding="utf-8")
#f.write("term_id,word\n")
#f.close()

In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

##### In this cell, we look for the words contained in the plot of each book. We remove from the plot the punctuation and the stopwords. We also stem the words ##### 

In [14]:
#set_of_words=set()
#for i in range(1,30001):
    #try:
        #s="tsv/article_"+str(i)+".tsv"
        #book=pd.read_csv(s,sep="\t")
        #data=book["Plot"][0]
        #words = word_tokenize(data)
        #words = [x.lower() for x in words if x.isalpha()]
        #stop_words = set(stopwords.words('english'))
        #words = [x for x in words if not x in stop_words]
        #porter = PorterStemmer()
        #words = [porter.stem(x) for x in words]
        #for elem in words:
            #set_of_words.add(elem)
    #except:
        #pass

##### In this cell, we add every word with the relative term_id to the vocabulary" #####

In [15]:
#n=0
#for i in set_of_words:
    #n=n+1
    #f = open("vocabulary.csv",'a',encoding="utf-8")
    #new_line=str(n)+", "+i+"\n"
    #f.write(new_line)
    #f.close()

##### In the next cells, we create the Inverted Index. It's a dictionary where the keys are the term_id, and the value of each key is a list of the id of each document whose plot contains all the searched words. We will save this dictionary in a .json file. #####

In [17]:
#inv_index={}
#for i in range (1,30001):
    #try:
        #s="tsv/article_"+str(i)+".tsv"
        #set_of_words=set()
        #book=pd.read_csv(s,sep="\t")
        #data=book["Plot"][0]
        #words = word_tokenize(data)
        #words = [x.lower() for x in words if x.isalpha()]
        #stop_words = set(stopwords.words('english'))
        #words = [x for x in words if not x in stop_words]
        #porter = PorterStemmer()
        #words = [porter.stem(x) for x in words]
        #for elem in words:
            #set_of_words.add(elem)
        #for y in set_of_words:
            #if y in inv_index:
                #inv_index[y].append(s)
            #else:
                #inv_index[y]=[s]
    #except:
        #pass

In [18]:
#data = pd.read_csv("vocabulary.csv")
#for index,row in data.iterrows():
    #numerical_key=row["term_id"]
    #word_as_key=row["word"].strip()
    #inv_index[numerical_key] = inv_index.pop(word_as_key.strip())

In [5]:
import json

In [6]:
#json.dump(inv_index,open("inv_index.json",'w'))

##### In this cell, we ask the user to write the words he is looking for. After executing the next cells, the user will receive as a result a dataframa containing some informations about the books whose plot contains all the words he looked for. #####

In [1]:
x=input("write the words you are looking for:")

write the words you are looking for:katniss, games


In [45]:
set_of_words=set()
words = word_tokenize(x)
words = [x.lower() for x in words if x.isalpha()]
stop_words = set(stopwords.words('english'))
words = [x for x in words if not x in stop_words]
porter = PorterStemmer()
words = [porter.stem(x) for x in words]
for elem in words:
    set_of_words.add(elem)

In [46]:
term_id_list=list()
for i in set_of_words:
    data = pd.read_csv("vocabulary.csv")
    for index,row in data.iterrows():
        if row["word;"].strip(" ;")==i:
            term_id_list.append(row["term_id"])
print(term_id_list)

[8713, 27386]


In [47]:
f=open("inv_index.json")
inv_index=json.load(f)
f.close()


In [48]:
books=set(inv_index[str(term_id_list[0])])
if len(term_id_list)>1:
    for i in term_id_list[1:]:
        key=str(i)
        new_set=set(inv_index[key])
        books=books.intersection(new_set)
books

{'tsv/article_223.tsv', 'tsv/article_325.tsv', 'tsv/article_5288.tsv'}

##### Here we can see the result of the search #####

In [56]:
title_list=list()
plot_list=list()
url_list=list()
for i in books:
    book=pd.read_csv(i,sep="\t")
    title_list.append(book["bookTitle"][0])
    plot_list.append(book["Plot"][0])
    url_list.append(book["Url"][0])
result=pd.DataFrame({"bookTitle":title_list,"Plot":plot_list,"Url":url_list})
result

,bookTitle,Plot,Url
0,"SAMPLER ONLY: Catching Fire (The Hunger Games,...","Against all odds, Katniss Everdeen has won th...",https://www.goodreads.com/book/show/20349441-...
1,Mockingjay,The final book in the ground-breaking HUNGER ...,https://www.goodreads.com/book/show/7260188-m...
2,Catching Fire,"Against all odds, Katniss Everdeen has surviv...",https://www.goodreads.com/book/show/6148028-c...
